In [140]:
import pandas as pd
import wikipedia
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import re

In [70]:
def bach(wik, page, head):
    cols = ['season', 'dates', 'bachelor', 'winner', 'runner-up', 'proposal', 'together', 'notes', a]
    bachelors = pd.DataFrame(columns=cols)
    url = wik + page.replace(' ', '_')
    req = urllib.request.urlopen(url)
    article = req.read().decode()
    data = []
    
    with open('ISO_3166-1_alpha-2.html', 'w', encoding='utf-8') as fo:
        fo.write(article)
        
    html = open('ISO_3166-1_alpha-2.html', encoding='utf-8').read()
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('table')
    
    for t in tables:
        ths = t.find_all('th')
        headings = [th.text.strip() for th in ths]
        
        try:
            if headings[:8] == head:
                with open('iso_3166-1_alpha-2_codes.txt', 'w', encoding='utf-8') as fo:
                    for tr in t.find_all('tr'):
                        tds = tr.find_all('td')
                        if not tds:
                            continue
                        data.append([td.text.strip() for td in tds[:8]])                      
                ret_df = pd.DataFrame(data, columns = cols)
                bachelors = bachelors.append(ret_df, sort=True)
                            
        except:
            ret_df = pd.DataFrame(columns = cols)

    bachelors = bachelors[bachelors['season'].apply(lambda x: x.isnumeric())]
    
    return bachelors

In [69]:
def cont(wik, page, head):
    cols = ['name', 'age', 'hometown', 'occupation', 'result'] 
    contestants = pd.DataFrame(columns = cols + ['season'])
    
    for p in range(len(page)):
        url = wik + page[p].replace(' ', '_')
        req = urllib.request.urlopen(url)
        article = req.read().decode()
        data = []

        with open('ISO_3166-1_alpha-2.html', 'w', encoding='utf-8') as fo:
            fo.write(article)

        html = open('ISO_3166-1_alpha-2.html', encoding='utf-8').read()
        soup = BeautifulSoup(html, 'html.parser')
        tables = soup.find_all('table')

        for t in tables:
            ths = t.find_all('th')
            headings = [th.text.strip() for th in ths]

            try:
                if headings[:3] == head:
                    with open('iso_3166-1_alpha-2_codes.txt', 'w', encoding='utf-8') as fo:
                        for tr in t.find_all('tr'):
                            tds = tr.find_all('td')
                            if not tds:
                                continue
                            data.append([td.text.strip() for td in tds[:5]])
                    ret_df = pd.DataFrame(data, columns = cols)
                    ret_df['season'] = page[p].split(' ')[3].split(')')[0]
                    contestants = contestants.append(ret_df, sort=True)
                            
            except:
                ret_df = pd.DataFrame(columns = cols + ['season'])
                contestants = contestants.append(ret_df, sort=True)
                
    return contestants

In [231]:
def age(page):
    keys = ['year-old', 'year old', 'years-old', 'years old']
    cols = ['season','age']
    ages = pd.DataFrame(columns=cols)
    data = []

    for p in page:
        content = wikipedia.WikipediaPage(title = p).summary
        kw = [k for k in keys if k in content]
        season = p.split(' ')[3].split(')')[0]

        if len(kw) != 0:
            if '-' in kw[0]:
                age = content[content.index(kw[0])-3:content.index(kw[0])+4].split('-')[0]
                data.append([season, age])
                ret_df = pd.DataFrame(data, columns=cols)
            else:
                age = content[content.index(kw[0])-3:content.index(kw[0])+4].split(' ')[0]
                data.append([season, age])
                ret_df = pd.DataFrame(data, columns=cols)
        else:
            age = None
            data.append([season, age])
            ret_df = pd.DataFrame(data, columns=cols)
            
    return ages

In [84]:
def rem_brack(d):
    return d.apply(lambda x: x.str.replace(r"\[.*\]",""))

In [67]:
#create dataframes            
wik = 'https://en.wikipedia.org/wiki/'
bach_home = 'The Bachelor (American TV series)'
bach_seasons = ['The Bachelor (season {})'.format(s) for s in list(range(1,25))]
ette_home = 'The Bachelorette'
ette_seasons = ['The Bachelorette (season {})'.format(s) for s in list(range(1,16))]
hd_bach = ['#', 'Original run', 'Bachelor', 'Winner', 'Runner(s)-up', 'Proposal', 'Still together', 'Relationship notes']
hd_ette = ['#', 'Original run', 'Bachelorette', 'Winner', 'Runner-up', 'Proposal', 'Still together', 'Relationship notes']
hd_cont = ['Name', 'Age', 'Hometown']

In [209]:
#bachelors
bachelors = bach(wik, bach_home, hd_bach)

In [59]:
#bachelor contestants
bach_contestants = cont(wik, bach_seasons, hd_cont)

In [71]:
#bachelorettes
ettes = bach(wik, ette_home, hd_ette)

In [73]:
#bachelorette contestants
ette_contestants = cont(wik, ette_seasons, hd_cont)

In [87]:
#add show type
bachelors['type'] = 'Bachelor'
bach_contestants['type'] = 'Bachelor'
ettes['type'] = 'Bachelorette'
ette_contestants['type'] = 'Bachelorette'

In [98]:
#remove hyperlinks and concatenate dataframes
leads = pd.DataFrame(columns=bachelors.columns)
contestants = pd.DataFrame(columns=bach_contestants.columns)
dfs = [bachelors, ettes, bach_contestants, ette_contestants]

for d in dfs:
    d = rem_brack(d)
    
    if 'result' in d.columns:
        contestants = contestants.append(d)
    else:
        leads = leads.append(d)